In [18]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import sys
import pandas as pd
import requests
from datetime import date
#from api_keys import AV_api, mk_token
AV_api = "1CSXU4M8SUPKSPG"

In [25]:
# define sectors
sectors = {
    'Semiconductors-Tech': ['NVDA', 'TSM', 'AVGO', 'AMD', 'INTC'],
    'Software': ['CRM', 'SAP', 'INTU', 'NOW', 'ZI'],
    'Consumer': ['AAPL', 'MSFT', 'SONY', 'LPL', 'SONO'],
    'Infotech': ['ACN', 'IBM', 'FI', 'INFY', 'CTSH']
}
sectors = pd.DataFrame(sectors)

In [6]:
all_data = []                                                                   # create an empty list to append to

start_date = '2018-01-01'                                                       # create start and end dates

end_date = '2023-12-04'

for industry, companies in sectors.items():                                       # iterate over each industry
    for ticker in companies:                                                    # iterate over each company in industry
        try:
            print (f'Gathering data for {industry}: {ticker}')
            
            ts = TimeSeries(key=AV_api, output_format='pandas')     # make api call
            data, meta_data = ts.get_daily(symbol=ticker, outputsize='full')
            data_df = pd.DataFrame(data)

            data_df = data_df.sort_index()                                      # filter from start to end dates
            filtered_data = pd.DataFrame(data_df.loc[start_date:end_date])

            #Ticker, sector, Industry, Open, High, Low, Close, Volume
            filtered_data["Ticker"] = ticker                                    # add ticker column
            filtered_data["Industry"] = industry
            filtered_data["Sector"] = 'Technology'
            all_data.append(filtered_data)
        except:
            print(f'Company {ticker} not found')

Gathering data for Semiconductors: NVDA
Gathering data for Semiconductors: TSM
Gathering data for Semiconductors: AVGO
Gathering data for Semiconductors: AMD
Gathering data for Semiconductors: INTC
Gathering data for Software: CRM
Gathering data for Software: SAP
Gathering data for Software: INTU
Gathering data for Software: NOW
Gathering data for Software: ZI
Gathering data for Consumer: AAPL
Gathering data for Consumer: MSFT
Gathering data for Consumer: SONY
Gathering data for Consumer: LPL
Gathering data for Consumer: SONO
Company SONO not found
Gathering data for Infotech: ACN
Gathering data for Infotech: IBM
Gathering data for Infotech: FI
Gathering data for Infotech: INFY
Gathering data for Infotech: CTSH


In [8]:
final_data = pd.concat(all_data)                        # Convert to dataframe
final_data.rename
final_data.to_csv('tech_data_test.csv')        # Save to csv

In [26]:
tech_data = pd.read_csv('tech_data_test.csv')
tech_data = tech_data.rename(columns={'1. open': 'Open', 
                                      '2. high' : 'High', 
                                      '3. low' : 'Low',
                                      '4. close': 'Close',
                                      '5. volume': 'Volume',
                                      'Companies': 'sector',
                                      'date' : "Date"})
tech_data
tech_data['sector'] = 'Technology'
tech_data = tech_data[['Date','Ticker', 'Sector', 'Industry', 'Open', 'High', 'Low', 'Close', 'Volume']]
tech_data.to_csv('tech_data.csv', index = False)
tech_data['Date'] = pd.to_datetime(tech_data['Date'])
tech_data.to_csv('tech_data.csv', index = False)
tech_data

,Date,Ticker,Sector,Industry,Open,High,Low,Close,Volume
0,2018-01-02,NVDA,Technology,Semiconductors,195.78,199.50,194.500,199.35,8303752.0
1,2018-01-03,NVDA,Technology,Semiconductors,204.10,213.70,203.750,212.47,22499518.0
2,2018-01-04,NVDA,Technology,Semiconductors,215.76,218.05,212.690,213.59,14346675.0
3,2018-01-05,NVDA,Technology,Semiconductors,214.19,216.91,211.080,215.40,14008768.0
4,2018-01-08,NVDA,Technology,Semiconductors,220.40,225.00,218.580,222.00,21190932.0
...,...,...,...,...,...,...,...,...,...
27715,2023-11-28,CTSH,Technology,Infotech,70.70,71.52,70.700,71.30,2607377.0
27716,2023-11-29,CTSH,Technology,Infotech,71.60,71.80,69.895,70.12,3521683.0
27717,2023-11-30,CTSH,Technology,Infotech,70.39,70.52,69.800,70.38,8254394.0
27718,2023-12-01,CTSH,Technology,Infotech,70.23,70.87,70.080,70.82,2672566.0
